In [43]:
import psycopg2
import pandas as pd
import datetime

In [44]:
# SET UP

In [71]:
con = psycopg2.connect(database="postgres", user="postgres", password="soaresmariana", host="127.0.0.1", port="5432")
cur = con.cursor()

In [4]:
df_booking = pd.read_csv('bookings.csv', sep=',')
df_maintenance = pd.read_csv('maintenance.csv', sep=',')

In [ ]:
# CREATE TABLES

In [ ]:
query = '''
        CREATE TABLE IF NOT EXISTS maintenance (
            id_maintenance int,
            id_apart int,
            start_date DATE,
            end_date DATE
            )
        '''
cur.execute(query)

In [ ]:
query = '''
        CREATE TABLE IF NOT EXISTS bookings (
            id_apart int,
            check_in DATE,
            check_out DATE,
            id_tenant SERIAL,
            tenant_name VARCHAR(255),
            email VARCHAR(255)
            )
        '''
cur.execute(query)

In [ ]:
query = '''
        CREATE TABLE IF NOT EXISTS calendar (
            dates Date,
            free_aparts int
            )
        '''
cur.execute(query)

In [ ]:
con.commit()

In [ ]:
# PREPARE AND LOAD DATA

In [ ]:
## MAINTENANCE

In [13]:
# df_maintenance['Start Date'] = pd.to_datetime(df_maintenance['Start Date']).dt.strftime("%m/%d/%Y")
# df_maintenance['End Date'] = pd.to_datetime(df_maintenance['End Date']).dt.strftime("%m/%d/%Y")
# cols = ','.join(list(df_maintenance.columns))
# values = list(map(tuple, df_maintenance.values))

In [9]:
## BOOKING

In [17]:
df_booking['Check-In'] = pd.to_datetime(df_booking['Check-In']).dt.strftime("%m/%d/%Y")
df_booking['Check-Out'] = pd.to_datetime(df_booking['Check-Out']).dt.strftime("%m/%d/%Y")
cols = ','.join(list(df_booking.columns))
values = list(map(tuple, df_booking.values))

In [11]:
## LOAD TABLES

In [18]:
values_bytes = b','.join(cur.mogrify("%s", (x, )) for x in values)
# query = b"INSERT INTO maintenance VALUES " + values_bytes
query = b"INSERT INTO bookings VALUES " + values_bytes
cur.execute(query)
con.commit()

In [ ]:
## TEMP_CALENDAR - BUSY APARTS

In [19]:
query = '''
        CREATE TEMP TABLE temp_calendar (
            dates Date,
            id_apart int,
            type VARCHAR(255)
            )
            ON COMMIT DROP
        '''
cur.execute(query)

In [ ]:
dates = pd.date_range(start='12/23/2020', end=datetime.datetime.strptime('09/02/2022', "%m/%d/%Y"), freq='D')

In [27]:
values = []
for date in dates:
    for b in df_booking.index:
        if (date >= datetime.datetime.strptime(df_booking['Check-In'][b], "%m/%d/%Y") and date <= datetime.datetime.strptime(df_booking['Check-Out'][b], "%m/%d/%Y")):
                values.extend([date.strftime('%m/%d/%Y'), df_booking['ID Apart'][b], 'Booking'])

In [29]:
for date in dates:
    for m in df_maintenance.index:
        if (date >= datetime.datetime.strptime(df_maintenance['Start Date'][m], "%m/%d/%Y") and date <= datetime.datetime.strptime(df_maintenance['End Date'][m], "%m/%d/%Y")):
                values.extend([date.strftime('%m/%d/%Y'), df_maintenance['ID Apart'][m], 'Maintenance'])

In [32]:
values_list = [values[i:i+3] for i in range(0, len(values), 3)]
fvalues = tuple(tuple(x) for x in values_list)

In [ ]:
## LOAD TEMP_CALENDAR

In [33]:
for f in fvalues:
    query = f"INSERT INTO temp_calendar VALUES {f}"
    cur.execute(query)

In [ ]:
# MAIN PROGRAM

In [34]:
df_aparts = pd.DataFrame({'id_aparts': df_booking['ID Apart'], 'type': 'Bookings'})
df_aparts2 = pd.DataFrame({'id_aparts': df_maintenance['ID Apart'], 'type': 'Maintanence'})
df_aparts=df_aparts.append(df_aparts2) 
total_aparts = int(len(df_aparts['id_aparts'].unique()))

In [ ]:
query = ''' 
        select dates, COUNT(id_apart) 
        from temp_calendar tc 
        group by dates 
        order by dates;
        '''
cur.execute(query)
df_calendar = cur.fetchall()

In [39]:
values = []
for col in df_calendar:
    values.extend([col[0].strftime('%m/%d/%Y'), total_aparts - col[1]])
values_list = [values[i:i+2] for i in range(0, len(values), 2)]
fvalues = tuple(tuple(x) for x in values_list)

In [38]:
for f in fvalues:
    query = f"INSERT INTO calendar VALUES {f}"
    cur.execute(query)
    
con.commit()
con.close()